# Instructions
1. Load the houseprices data from Thinkful's database.
2. Do data cleaning, exploratory data analysis, and feature engineering. You can use your previous work in this module. But make sure that your work is satisfactory.
3. Now, split your data into train and test sets where 20% of the data resides in the test set.
4. Build several linear regression models including Lasso, Ridge, or ElasticNet and train them in the training set. Use k-fold cross-validation to select the best hyperparameters if your models include one!
5. Evaluate your best model on the test set.
6. So far, you have only used the features in the dataset. However, house prices can be affected by many factors like economic activity and the interest rates at the time they are sold. So, try to find some useful factors that are not included in the dataset. Integrate these factors into your model and assess the prediction performance of your model. Discuss the implications of adding these external variables into your model.


### Start Notebook environment

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

In [2]:
# %load ../utility/overhead.py
#record module versions used in cell 1
#
def version_recorder():
    '''
    only works if import is first cell run. prints and then returns dictionary with modules:version.
    '''
    import pkg_resources
    resources = In[1].splitlines()
    ##ADD: drop lines if not _from_ or _import_
    version_dict = { resource.split()[1].split(".")[0] : pkg_resources.get_distribution(resource.split()[1].split(".")[0]).version for resource in resources }
    return version_dict
version_recorder()

{'numpy': '1.16.2',
 'pandas': '0.24.2',
 'sklearn': '0.0',
 'matplotlib': '3.0.3',
 'seaborn': '0.9.0',
 'sqlalchemy': '1.3.5'}

# Data Cleaning

In [3]:
#credentials
user = 'dsbc_student'
pw = '7*.8G9QH21'
host = '142.93.121.174'
port = '5432'
db = 'houseprices'
dialect = 'postgresql'

db_addr = f'{dialect}://{user}:{pw}@{host}:{port}/{db}'
engine = create_engine(db_addr)

query = '''
SELECT
    *
FROM
    houseprices
'''

raw_df = pd.read_sql(query, con=engine)
engine.dispose()

In [4]:
house_df = raw_df.copy(deep=True)
house_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [7]:
house_df.columns

Index(['id', 'mssubclass', 'mszoning', 'lotfrontage', 'lotarea', 'street',
       'alley', 'lotshape', 'landcontour', 'utilities', 'lotconfig',
       'landslope', 'neighborhood', 'condition1', 'condition2', 'bldgtype',
       'housestyle', 'overallqual', 'overallcond', 'yearbuilt', 'yearremodadd',
       'roofstyle', 'roofmatl', 'exterior1st', 'exterior2nd', 'masvnrtype',
       'masvnrarea', 'exterqual', 'extercond', 'foundation', 'bsmtqual',
       'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfinsf1',
       'bsmtfintype2', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'heating',
       'heatingqc', 'centralair', 'electrical', 'firstflrsf', 'secondflrsf',
       'lowqualfinsf', 'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath',
       'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'kitchenqual',
       'totrmsabvgrd', 'functional', 'fireplaces', 'fireplacequ', 'garagetype',
       'garageyrblt', 'garagefinish', 'garagecars', 'garagearea', 'garagequal',
       'garagecond', 'paved

In [8]:
#create df with categorical variables to select some features from
categorical_feat = house_df.dtypes[house_df.dtypes == 'object'].index
new_categories_df = house_df[["saleprice"]]
for feature in categorical_feat:
    new_categories_df = pd.concat([new_categories_df, 
                                   pd.get_dummies(house_df[feature], columns=categorical_feat, drop_first=True, prefix = feature)], axis=1)

In [9]:
new_categories_df.corr()[["saleprice"]].sort_values(by='saleprice')

,saleprice
exterqual_TA,-0.589044
kitchenqual_TA,-0.519298
bsmtqual_TA,-0.452394
garagefinish_Unf,-0.410608
masvnrtype_None,-0.374468
garagetype_Detchd,-0.354141
foundation_CBlock,-0.343263
heatingqc_TA,-0.312677
mszoning_RM,-0.288065
lotshape_Reg,-0.267672


In [10]:
#append numerical features to new df
new_categories_df = pd.concat([new_categories_df, 
                               house_df.filter(items=(house_df.columns[(house_df.dtypes.values != 'object').tolist()]), axis=1) ], 
                              axis=1) #tolist() needed to avoid hashability issue

#create X & y
X = house_df[["overallqual", "grlivarea", "fullbath", "yearbuilt",]]
X = pd.concat([X, new_categories_df[['exterqual_TA', 'foundation_CBlock']]], 1)
X["overalqual_x_year"] = house_df.overallqual * house_df.yearbuilt
y = house_df.saleprice

# Model Comparison
## Split data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.15)

## Lasso

In [12]:
lasso_model = Lasso(tol=.5)  #increased tol to avoid 'ConvergenceWarning: Objective did not converge'
alphas = np.array([.0001, .001, .01, .1, 1, 10, 100, 1_000, 10_000, 100_000])

grid_las = GridSearchCV(estimator=lasso_model, param_grid=dict(alpha=alphas), cv=12)
grid_las.fit(X_train, y_train)
print(grid_las)
print("best r^2 is: {}".format(grid_las.best_score_))
print("associated lambda value is: {}".format(grid_las.best_estimator_.alpha))

GridSearchCV(cv=12, error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.5, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04, 1.e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
best r^2 is: 0.7907816472650003
associated lambda value is: 10000.0


c:\users\chris\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


## Ridge

In [13]:
ridge_model = Ridge()
alphas = np.array([.0001, .001, .01, .1, 1, 10, 100, 1_000, 10_000, 100_000])

grid_ridge = GridSearchCV(estimator=ridge_model, param_grid=dict(alpha=alphas), cv=7)
grid_ridge.fit(X_train, y_train)
print(grid_ridge)
print("best r^2 is: {}".format(grid_ridge.best_score_))
print("associated lambda value is: {}".format(grid_ridge.best_estimator_.alpha))

GridSearchCV(cv=7, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04, 1.e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
best r^2 is: 0.8148463166115153
associated lambda value is: 0.01


c:\users\chris\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


## ElasticNet

In [14]:
elasticnet_model = ElasticNet(tol=1)   #increased tol to avoid 'ConvergenceWarning: Objective did not converge'
alphas = np.array([.0001, .001, .01, .1, 1, 10, 100])
l1_ratio = np.array([.1, .5, .9])
params = {'alpha': alphas, 'l1_ratio': l1_ratio}

grid_enet = GridSearchCV(estimator=elasticnet_model, param_grid=params, cv=7)
grid_enet.fit(X_train, y_train)
print(grid_enet)
print("best r^2 is: {}".format(grid_enet.best_score_))
print("associated lambda value is: {}".format(grid_enet.best_estimator_.alpha))
print("associated estimator value is: {}".format(grid_enet.best_estimator_))

GridSearchCV(cv=7, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic', tol=1,
                                  warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'l1_ratio': array([0.1, 0.5, 0.9])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
best r^2 is: 0.7956667773823641
associated lambda value is: 100.0
associated estimator value is: ElasticNet(alpha=100.0, copy_X=True, fit_intercept=True, l1_ratio=0.9,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, s

c:\users\chris\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


# CV with test data

# External Factors

Realtors may cite Location, Location, Location, but timing is vital when buying and selling assets. The Great Recession had a huge impact on the housing market (and vice-versa.) I am including a feature for the economic condition: annual unemployment rate (monthly is available, but would introduce difficulties in comparing seasonality for house sales vs employment)

The largest problem with adding this is that it requires finding a separate data set and integrating that information into the extant data in my notebook. One particular issue is that while I found several ways to identify the end of the recession (classically, defined as 2 consecutive quarters), house prices take longer to recover and so there is no clear-cut way to define this term. Because such large-scale macroeconomic factors occur rarely, it is not easy to do testing to avoid overfitting; this I have tried to make the use of this term conservative.

Organization for Economic Co-operation and Development, Unemployment Rate: Aged 15-64: All Persons for the United States [LRUN64TTUSA156S], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/LRUN64TTUSA156S, August 6, 2019.

In [15]:
econ_df = pd.read_csv(r'../../../data_sets/LRUN64TTUSA156S.csv')

In [16]:
econ_df.DATE = pd.to_datetime(econ_df["DATE"])

In [20]:
econ_df["year"] = econ_df.DATE.dt.year
econ_df.head()

,DATE,LRUN64TTUSA156S,year
0,1960-01-01,5.617115,1960
1,1961-01-01,6.770271,1961
2,1962-01-01,5.596210,1962
3,1963-01-01,5.739126,1963
4,1964-01-01,5.246175,1964


In [21]:
house_w_ue_df = pd.merge(left=house_df, right=econ_df, left_on='yrsold', right_on='year', how='left')
X_weu = pd.concat([X, house_w_ue_df.LRUN64TTUSA156S])

In [22]:
elasticnet_model2 = ElasticNet(tol=1)   #increased tol to avoid 'ConvergenceWarning: Objective did not converge'
alphas = np.array([.0001, .001, .01, .1, 1, 10, 100])
l1_ratio = np.array([.1, .5, .9])
params = {'alpha': alphas, 'l1_ratio': l1_ratio}

grid_enet2 = GridSearchCV(estimator=elasticnet_model2, param_grid=params, cv=7)
grid_enet.fit(X_train, y_train)
print(grid_enet)
print("best r^2 is: {}".format(grid_enet.best_score_))
print("associated lambda value is: {}".format(grid_enet.best_estimator_.alpha))
print("associated estimator value is: {}".format(grid_enet.best_estimator_))

GridSearchCV(cv=7, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic', tol=1,
                                  warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'l1_ratio': array([0.1, 0.5, 0.9])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
best r^2 is: 0.7956667773823641
associated lambda value is: 100.0
associated estimator value is: ElasticNet(alpha=100.0, copy_X=True, fit_intercept=True, l1_ratio=0.9,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, s

c:\users\chris\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
